In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PyCO2SYS as pyco2

pyco2.__version__

'1.8.1'

## add the CANYON-B results to the existing dataframe

In [2]:
df = pd.read_csv("data/in_situ_rates_compiled_withWOA.csv")
cb = pd.read_csv("data/compiled_WOA_CANYONBresults.csv")

df["TA_cb"] = cb['Alkalinity']
df["DIC_cb"] = cb['DIC']
df["NO3_cb"] = cb["Nitrate"]
df["PO4_cb"] = cb["Phosphate"]
df["SiOH4_cb"] = cb["Silicate"]
df["pH_insitu_cb"] = cb["pH"]

## PyCO2Sys

In [3]:
#do pyco2sys
result_calc23 = pyco2.sys(par1=df["DIC_cb"], par2=df["pH_insitu_cb"], par1_type = 2, par2_type=3, 
                   salinity=df["Sal_woa"], temperature=df["Temp_woa"], pressure=df["Pressure"],
                   total_silicate=df["SiOH4_cb"], total_phosphate=df["PO4_cb"])


#add the columns that I find necessary
new_cols = ["k_calcite_calc23", "Oca_calc23", "k_aragonite_calc23", "Oar_calc23", 
           "total_calcium_calc23", "CO2_calc23", "HCO3_calc23", "CO3_calc23"]
col = ["k_calcite", "saturation_calcite", "k_aragonite", "saturation_aragonite", 
       "total_calcium", "CO2", "HCO3", "CO3"]
df[new_cols] = pd.DataFrame(result_calc23)[col]

#make an extra column that is just "Omega" 
#which either takes the saturation_calcite or saturation_aragonite value, depending on material type
df["Omega_calc23"] = np.where(df["Material"].eq("Aragonite"), df["Oar_calc23"], df["Oca_calc23"])
#except for nan or Mg Calcite
df.loc[((df["Material"] == "Mg-Calcite") | (pd.isna(df["Material"]))), "Omega_calc23"] = np.nan
df.loc[((df["Material"] == "Silica") | (pd.isna(df["Material"]))), "Omega_calc23"] = np.nan


### calculate the metabolic saturation state, based on Subhas et al. (2022)

this is with TA and DIC as parameters (and not DIC and pH) because the equations in the paper recalculate those

In [4]:
dic_metab = df['DIC_cb'] + df['Doxy_woa'] * 0.688
ta_metab = df['TA_cb'] - df['Doxy_woa'] * 0.0941
phos_metab = df['PO4_cb'] + df['Doxy_woa'] * 0.0059

#do pyco2sys
result_metab = pyco2.sys(par1=ta_metab, par2=dic_metab, par1_type = 1, par2_type=2, 
                   salinity=df["Sal_woa"], temperature=df["Temp_woa"], pressure=df["Pressure"],
                   total_silicate=df["SiOH4_cb"], total_phosphate=phos_metab)

df["Oca_metab_calc12"] = result_metab['saturation_calcite']
df["Oar_metab_calc12"] = result_metab['saturation_aragonite']


#make an extra column that is just "Omega" 
#which either takes the saturation_calcite or saturation_aragonite value, depending on material type
df["Omega_metab_calc12"] = np.where(df["Material"].eq("Aragonite"), df["Oar_metab_calc12"], 
                             df["Oca_metab_calc12"])

#except for nan or Mg Calcite
df.loc[((df["Material"] == "Mg-Calcite") | (pd.isna(df["Material"]))), "Omega_metab_calc12"] = np.nan
df.loc[((df["Material"] == "Silica") | (pd.isna(df["Material"]))), "Omega_metab_calc12"] = np.nan


## save resulting dataframe

In [5]:
df.to_csv("data/in_situ_rates_compiled_allvar.csv", index=False)